In [3]:
def calculate_iou(box_a, box_b):
    """
    Berekent de Intersection over Union (IoU) tussen twee bounding boxes.
    
    Parameters:
        box_a (list or tuple): Bounding box in [x, y, w, h]-formaat.
        box_b (list or tuple): Bounding box in [x, y, w, h]-formaat.
    
    Returns:
        float: IoU-waarde tussen 0.0 en 1.0.
    """
    # Zet om naar [x1, y1, x2, y2]
    x1_a, y1_a, w_a, h_a = box_a
    x2_a, y2_a = x1_a + w_a, y1_a + h_a

    x1_b, y1_b, w_b, h_b = box_b
    x2_b, y2_b = x1_b + w_b, y1_b + h_b

    # Bereken overlapcoördinaten
    x_left = max(x1_a, x1_b)
    y_top = max(y1_a, y1_b)
    x_right = min(x2_a, x2_b)
    y_bottom = min(y2_a, y2_b)

    # Check of er overlap is
    if x_right < x_left or y_bottom < y_top:
        return 0.0  # Geen overlap

    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box_a_area = w_a * h_a
    box_b_area = w_b * h_b
    union_area = box_a_area + box_b_area - intersection_area

    return intersection_area / union_area

In [4]:
def check(condition, message):
    """
    Controleert een conditie en geeft een foutmelding als de conditie niet waar is.
    
    Parameters:
        condition (bool): De te controleren conditie.
        message (str): Foutmelding die wordt weergegeven als de conditie niet waar is.
    
    Returns:
        bool: True als de conditie waar is, False anders.
    """
    if not condition:
        print(f"Test mislukt: {message}")
        return False
    return True


def test_calculate_iou_verbose():
    """
    Voert uitgebreide tests uit voor de calculate_iou functie.
    Geeft een samenvatting van alle testresultaten.
    """
    all_ok = True

    # Test 1: Geen overlap
    box_a = [0, 0, 2, 2]
    box_b = [3, 3, 2, 2]
    all_ok &= check(calculate_iou(box_a, box_b) == 0.0, "Test 1 mislukt: geen overlap")

    # Test 2: Volledige overlap (identieke boxen)
    box_a = [1, 1, 2, 2]
    box_b = [1, 1, 2, 2]
    all_ok &= check(calculate_iou(box_a, box_b) == 1.0, "Test 2 mislukt: volledige overlap")

    # Test 3: Gedeeltelijke overlap
    box_a = [0, 0, 4, 4]
    box_b = [2, 2, 4, 4]
    iou = calculate_iou(box_a, box_b)
    expected_iou = 4 / (16 + 16 - 4)  # overlap van 2x2
    all_ok &= check(abs(iou - expected_iou) < 1e-6, "Test 3 mislukt: gedeeltelijke overlap")

    # Test 4: Box binnen andere box
    box_a = [0, 0, 5, 5]
    box_b = [1, 1, 2, 2]
    iou = calculate_iou(box_a, box_b)
    expected_iou = (2 * 2) / (25)  # overlap is box_b, union is box_a
    all_ok &= check(abs(iou - expected_iou) < 1e-6, "Test 4 mislukt: box binnen andere")

    # Test 5: Box met nul breedte of hoogte
    box_a = [0, 0, 0, 2]
    box_b = [0, 0, 2, 2]
    all_ok &= check(calculate_iou(box_a, box_b) == 0.0, "Test 5 mislukt: nul breedte")

    box_a = [0, 0, 2, 0]
    all_ok &= check(calculate_iou(box_a, box_b) == 0.0, "Test 6 mislukt: nul hoogte")

    # Test 6: Rand-aan-rand contact (geen overlap)
    box_a = [0, 0, 2, 2]
    box_b = [2, 0, 2, 2]
    all_ok &= check(calculate_iou(box_a, box_b) == 0.0, "Test 7 mislukt: randcontact")

    # Test 7: Decimalen als coördinaten
    box_a = [0.5, 0.5, 1.5, 1.5]
    box_b = [1, 1, 2, 2]
    iou = calculate_iou(box_a, box_b)
    expected_iou = 1.0 / (2.25 + 4 - 1.0)
    all_ok &= check(abs(iou - expected_iou) < 1e-6, "Test 8 mislukt: decimalen")

    if all_ok:
        print("Alle uitgebreide tests zijn geslaagd!")
    else:
        print("Sommige tests zijn mislukt!")


# Voer de test uit
test_calculate_iou_verbose()

Alle uitgebreide tests zijn geslaagd!


In [5]:
IOU_THRESHOLD = 0.5

def match_detections_to_gt(image, gt_annotations, detection_data):
    """
    Vergelijkt gedetecteerde objecten met ground-truth annotaties voor een afbeelding.
    
    Parameters:
        image (dict): Metadata van de afbeelding, bevat 'id' en 'file_name'.
        gt_annotations (list): Lijst met ground-truth annotaties in COCO-formaat.
        detection_data (list): Lijst met detectie-uitvoer, met bounding boxes en labels.
    
    Returns:
        dict: Informatie over het aantal grondwaarheden, detecties en correcte matches.
    """
    image_id = image["id"]
    file_name = image["file_name"]

    # Filter ground-truth bboxes met category_id = 4 (Raam)
    gt_boxes = [
        ann["bbox"]
        for ann in gt_annotations
        if ann["image_id"] == image_id and ann["category_id"] == 4
    ]

    # Filter detecties: alleen objecten met label 'Raam' of 'Deur'
    det_boxes = [
        det["bounding_box"]
        for det in detection_data
        if det["label"] in ["Raam", "Deur"]
    ]

    matched_gt = set()
    correct_detections = 0

    for det_box in det_boxes:
        for i, gt_box in enumerate(gt_boxes):
            if i in matched_gt:
                continue

            iou = calculate_iou(det_box, gt_box)

            if iou >= IOU_THRESHOLD:
                correct_detections += 1
                matched_gt.add(i)
                break  # Stop met zoeken zodra match is gevonden

    return {
        "file_name": file_name,
        "gt_count": len(gt_boxes),
        "det_count": len(det_boxes),
        "correct": correct_detections
    }

In [6]:
def check(condition, message):
    """
    Controleert een conditie en geeft een foutmelding als de conditie niet waar is.
    
    Parameters:
        condition (bool): De te controleren conditie.
        message (str): Foutmelding die wordt weergegeven als de conditie niet waar is.
    
    Returns:
        bool: True als de conditie waar is, False anders.
    """
    if not condition:
        print(f"Test mislukt: {message}")
        return False
    return True


def test_match_detections_to_gt_verbose():
    """
    Voert uitgebreide tests uit voor de match_detections_to_gt functie.
    Test verschillende scenario's inclusief matching, filtering en tellingen.
    """
    # Testdata opzetten
    image = {"id": 1, "file_name": "test.jpg"}
    gt_annotations = [
        {"image_id": 1, "category_id": 4, "bbox": [0, 0, 2, 2]},   # Raam 1
        {"image_id": 1, "category_id": 4, "bbox": [3, 3, 2, 2]},   # Raam 2
        {"image_id": 1, "category_id": 5, "bbox": [5, 5, 2, 2]},   # Deur (wordt genegeerd)
    ]
    detection_data = [
        {"label": "Raam", "bounding_box": [0, 0, 2, 2]},   # Match raam 1
        {"label": "Deur", "bounding_box": [5, 5, 2, 2]},   # Deur detectie
        {"label": "Raam", "bounding_box": [10, 10, 2, 2]}, # Geen match
    ]

    # Voer de functie uit
    resultaat = match_detections_to_gt(image, gt_annotations, detection_data)

    # Voer alle checks uit
    tests_ok = True
    tests_ok &= check(
        resultaat["file_name"] == "test.jpg", 
        "file_name klopt niet"
    )
    tests_ok &= check(
        resultaat["gt_count"] == 2, 
        f"gt_count is {resultaat['gt_count']} ipv 2"
    )
    tests_ok &= check(
        resultaat["det_count"] == 3, 
        f"det_count is {resultaat['det_count']} ipv 3"
    )
    tests_ok &= check(
        resultaat["correct"] == 1, 
        f"correct is {resultaat['correct']} ipv 1"
    )

    # Rapporteer het eindresultaat
    if tests_ok:
        print("Alle tests geslaagd!")
    else:
        print("Sommige tests zijn mislukt!")


# Voer de test uit
test_match_detections_to_gt_verbose()

Alle tests geslaagd!
